##Waste Data Over Time

Imran Malek - imran@imranmalek.com

Looking at Waste Data and Oil Prices

Waste Data sourced from the US EPA

Consumption Data sourced from InflationData.com - http://inflationdata.com/Inflation/Inflation_Rate/Historical_Oil_Prices_Table.asp

In [115]:
import requests,pandas as pd, plotly.plotly as py,json,re
from plotly import tools
from plotly.graph_objs import *

def ConvertForPlots(dataframe,index_name,new_index_name): #Modified ConvertForPlots from previous notebook at https://github.com/cximran/Meat-Consumption-and-Agricultural-Stats/blob/master/Meat%20Consumption%20%26%20Land%20Stats.ipynb
    dataframe.fillna('',inplace=True)
    dataframe=dataframe.set_index(index_name).T
    dataframe.reset_index(inplace=True)
    dataframe.rename(columns={dataframe.columns[0]:'Year'},inplace=True)
    dataframe.index.name=index_name
    return(dataframe)
waste_data=pd.read_excel("FH - Materials Generated.xlsx")
waste_data.drop(waste_data.columns[[6,7,9,10]],1,inplace=True)
crude_oil_data=ConvertForPlots(pd.read_csv("crude_oil_prices.csv"),'Year','Price')

In [116]:
#Applying the colors to be used in the plot, sourced colors from http://www.tayloredmktg.com/rgb/

colors={}
colors['Paper and Paperboard']='#ffe4b5'
colors['Glass']='#f0f8ff'
colors['Metals']='#bebebe'
colors['Plastics']='#eeeee0'
colors['Rubber and Leather']='#cdb79e'
colors['Textiles']='#ffc0cb'
colors['Wood']='#deb887'
colors['Other']='#d8bfd8'
colors['Food Waste']='#ff0000'
colors['Yard Trimmings']='#3cb371'
colors['Misc. Inorganic Wastes']='#483d8b'

In [117]:
waste_data=waste_data.set_index('Materials')#Setting the index to Materials for easy calculations on the data itself 

In [118]:
#Populating the data traces inside of the bar graph
wastetraces=[]
for row in waste_data.index:
    if row=='Food Waste': #setting a different legend group to specifically call out food waste 
        legend_group='AFood'
    else:
        legend_group='Everything Else'
    wastetraces.append(
        Bar(
        x=waste_data.columns,
        y=waste_data.loc[row].values,
        name=row,
        marker=Marker(color=colors[row.strip()],line=Line(width=0)),
        legendgroup=legend_group
   
        )
    
    )
wastedata=Data(wastetraces)

In [189]:
waste_composition_figure=Figure(data=wastedata,layout=Layout(barmode='stack',barnorm='percent',title='Waste in America - composition',yaxis=YAxis(tickformat='{:.2f}',hoverformat='.2f',title='% composition')))

In [187]:
#Generating the data for food waste and oil prices over time

food_waste_time=Figure(data=[Scatter(
                                        x=waste_data.columns,
                                        y=waste_data.loc['Food Waste'].values,
                                        name='Food Waste',
                                        marker=Line(color='#ff0000')),
        
                            Scatter(
                                        x=waste_data.columns,
                                        y=[crude_oil_data[x][1] for x in waste_data.columns],
                                        name='Crude Oil Price',
                                        yaxis='y2',
                                        marker=Line(color='#000000')
                                )],
                       layout=Layout(title='American food waste and oil prices',yaxis=YAxis(side='left',title='Food Waste (in thousands of tons)'),yaxis2=YAxis(side='right',overlaying='y',title='Crude oil price per barrel (inflation adjusted))',hoverformat='$.2f')))


In [181]:
#Populating the data traces inside of the line graph over time
 wastelinetraces=[]
for row in waste_data.index:
    if row=='Food Waste': #setting a different legend group to specifically call out food waste 
        legend_group='AFood'
    else:
        legend_group='Everything Else'
    wastelinetraces.append(
        Scatter(
        x=waste_data.columns,
        y=waste_data.loc[row].values,
        name=row,
        marker=Marker(color=colors[row.strip()],line=Line(width=0)),
        legendgroup=legend_group
   
        )
    
    )
wastelinedata=Data(wastelinetraces)
waste_line_chart=Figure(data=wastelinedata,layout=Layout(title='Waste in America - over time',yaxis=YAxis(tickformat='{:.2f}',hoverformat='.2f',title='Amount of Waste (in thousands of tons)')))

In [186]:
py.iplot(waste_line_chart,filename='Food Waste in America over time')
py.iplot(waste_composition_figure,filename='Trash Composition in the US over time')
py.iplot(food_waste_time,filename='Food Waste & Oil Prices over time in the US')